In [ ]:
# General imports
import numpy as np
import torch
import random

# DeepMoD functions
from deepymod import DeepMoD
from deepymod.model.func_approx import NN
from deepymod.model.library import Library2D_third
from deepymod.model.constraint import LeastSquares
from deepymod.model.sparse_estimators import Threshold,PDEFIND
from deepymod.training import train
from deepymod.training.sparsity_scheduler import TrainTestPeriodic
from scipy.io import loadmat

# Settings for reproducibility
np.random.seed(1)
torch.manual_seed(1)

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [ ]:
for idt in [1,2,3,4,5,7,10]:
    
    data = np.load('diffusion_advection_29longb.npy').T
    data = data[120:-60,:,5:30]
    width, width_2, steps = data.shape
    data= np.take(data,np.arange(0,steps,idt),axis=2)
    width, width_2, steps = data.shape
    x_arr, y_arr, t_arr = np.linspace(0,1,width), np.linspace(0,1,width_2), np.linspace(0,1,steps)
    x_grid, y_grid, t_grid = np.meshgrid(x_arr, y_arr, t_arr, indexing='ij')


    num_sam_x, num_sam_y = 8,10
    tot_samples = num_sam_x*num_sam_y
    x_grid_new = np.empty([num_sam_x,num_sam_y,steps])
    y_grid_new = np.empty([num_sam_x,num_sam_y,steps])
    t_grid_new = np.empty([num_sam_x,num_sam_y,steps])
    data_new = np.empty([num_sam_x,num_sam_y,steps])
    for i in np.arange(steps):
        list_x = [random.sample(range(1,width),num_sam_y) for i in np.arange(0,num_sam_x)]
        list_y = [random.sample(range(1,width_2),num_sam_y) for i in np.arange(0,num_sam_x)]
        x_grid_new[:,:,i] = x_grid[list_x,list_y,i]
        y_grid_new[:,:,i] = y_grid[list_x,list_y,i]
        t_grid_new[:,:,i] = t_grid[list_x,list_y,i]
        data_new[:,:,i] = data[list_x,list_y,i]
    print("Dowmsampled shape:",x_grid_new.shape, "Total number of data points:", np.product(x_grid_new.shape))

    X = np.transpose((t_grid_new.flatten(), x_grid_new.flatten(), y_grid_new.flatten()))
    y = np.float32(data_new.reshape((data_new.size, 1)))
    y = y/np.max(y)
    # Add noise 
    noise_level = 0.0
    y_noisy = y + noise_level * np.std(y) * np.random.randn(y.size, 1)
    # Randomize data 

    idx = np.random.permutation(y.shape[0])
    X_train = torch.tensor(X[idx, :], dtype=torch.float32, requires_grad=True).to(device)
    y_train = torch.tensor(y_noisy[idx], dtype=torch.float32).to(device)

    # Configure DeepMoD

    network = NN(3, [40, 40, 40, 40], 1)
    library = Library2D_third(poly_order=0) 
    estimator = Threshold(0.05) 
    sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
    constraint = LeastSquares() 
    model = DeepMoD(network, library, estimator, constraint).to(device)
    optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=2e-3) 
    logdir='random_x_8_10/t_' + str(data.shape[2])
    train(model, X_train, y_train, optimizer,sparsity_scheduler, log_dir=logdir, split=0.8, max_iterations=50000, delta=1e-5, patience=200) 

Dowmsampled shape: (8, 10, 25) Total number of data points: 2000
  2100  MSE: 6.35e-04  Reg: 2.15e-04  L1: 1.06e+04 